In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
from sklearn.linear_model import LogisticRegression

In [3]:
train_data = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\train.csv')
x_test = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\test.csv')
x_train = train_data.drop(columns='TARGET')
y_train = train_data['TARGET'].astype('float')


In [4]:
x_test = x_test.set_index('SK_ID_CURR')
x_train = x_train.set_index('SK_ID_CURR')
submit = pd.DataFrame(x_test.index)

In [5]:

# numeric_transformer = Pipeline(steps= [
#         ('imputer', SimpleImputer(strategy='median')),
#         ('power_transformer', PowerTransformer(method='yeo-johnson')),
#         ('mmsc', MinMaxScaler(feature_range=(0,1)))

# ])

# categorical_transformer = Pipeline(steps = [
#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('onehot', OneHotEncoder(handle_unknown='ignore')),
# ])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_columns),
#         ('cat', categorical_transformer, categorical_columns)
#     ],remainder='passthrough')

# # Create a pipeline with preprocessing and LogisticRegression
# pipeline = Pipeline(steps= [
#     ('preprocess',preprocessor)
# ])

# # Fit the pipeline on training data
# x_train = (pipeline.fit_transform(x_train)).toarray()
# x_train = pd.DataFrame(x_train)



### Predict

In [6]:
model = LogisticRegression(penalty='elasticnet', l1_ratio=0.6, solver='saga')
model.fit(x_train,y_train)
y_pred = model.predict_proba(x_test)[:, 1]


d:\Python\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [7]:
submit['TARGET'] = y_pred
submit = submit.set_index('SK_ID_CURR')
submit = submit.sort_index(ascending=True)
submit.to_csv('../Submission.csv')
